## Books Recommender System using CLustering | Collaborative Based Filtering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
books = pd.read_csv('data/Books.csv', encoding='latin-1')

C:\Users\Mukand\AppData\Local\Temp\ipykernel_14196\2396178895.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/Books.csv', encoding='latin-1')


In [3]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [4]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [5]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [6]:
books.rename(columns={"Book-Title":"Title",
                     "Book-Author": 'Author',
                     'Year-Of-Publication':'Year',
                     'Image-URL-L':'Img-URL'},
            inplace=True)

C:\Users\Mukand\AppData\Local\Temp\ipykernel_14196\66292342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.rename(columns={"Book-Title":"Title",


In [7]:
books.shape

(271360, 6)

In [8]:
users = pd.read_csv('data/Users.csv', encoding='latin-1')

In [9]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
users.rename(columns={'User-ID':'ID'}, inplace=True)

In [11]:
ratings = pd.read_csv('data/Ratings.csv', encoding='latin-1')

In [12]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,155061224.00,5
2,276727,446520802.00,0
3,276729,052165615X,3
4,276729,521795028.00,6


In [13]:
ratings.rename(columns={'User-ID':'ID', 'Book-Rating':'Rating'}, inplace=True)

shapes

In [14]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 6)
(1048575, 3)
(278858, 3)


In [15]:
ratings['ID'].value_counts()

ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
104999        1
105002        1
105008        1
105014        1
123969        1
Name: count, Length: 95513, dtype: int64

In [16]:
ratings['ID'].unique().shape

(95513,)

#### those users who has rated more than 300 books

In [17]:
x=ratings['ID'].value_counts()>300
x[x].shape

(506,)

In [18]:
y=x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       124078, 224646,  85701,  63394,  15418, 106816,  92853,  82511,  62895,
        37567],
      dtype='int64', name='ID', length=506)

In [19]:
ratings = ratings[ratings['ID'].isin(y)]

In [20]:
ratings.shape

(407241, 3)

In [21]:
ratings_with_books = ratings.merge(books, on="ISBN")

In [117]:
ratings_with_books.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [22]:
ratings_with_books.shape

(31562, 8)

In [23]:
num_rating = ratings_with_books.groupby('Title')['Rating'].count().reset_index()

In [24]:
num_rating.head()

,Title,Rating
0,Earth Prayers From around the World: 365 Pray...,1
1,Flight of Fancy: American Heiresses (Zebra Ba...,1
2,Little Comic Shop of Horrors (Give Yourself G...,2
3,"Q-Zone (Star Trek The Next Generation, Book 48)",4
4,"!Trato hecho!: Spanish for Real Life, Combined...",1


In [25]:
num_rating.rename(columns={'Rating':'Num_of_rating'}, inplace=True)

In [26]:
num_rating

,Title,Num_of_rating
0,Earth Prayers From around the World: 365 Pray...,1
1,Flight of Fancy: American Heiresses (Zebra Ba...,1
2,Little Comic Shop of Horrors (Give Yourself G...,2
3,"Q-Zone (Star Trek The Next Generation, Book 48)",4
4,"!Trato hecho!: Spanish for Real Life, Combined...",1
...,...,...
12438,the Dark Light Years,1
12439,together by christmas,6
12440,voyage fatal,1
12441,why I'm like this : True Stories,1


In [27]:
final_rating = ratings_with_books.merge(num_rating, on ='Title')

In [28]:
final_rating.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL,Num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,59
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,59
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,59
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,59
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,59


In [29]:
final_rating.shape

(31562, 9)

#### book which got more than 70 ratings

In [30]:
final_rating = final_rating[final_rating['Num_of_rating']>=70]

In [31]:
final_rating.head()

,ID,ISBN,Rating,Title,Author,Year,Publisher,Img-URL,Num_of_rating
506,277427,080410753X,0,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,http://images.amazon.com/images/P/080410753X.0...,74
507,254,080410753X,0,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,http://images.amazon.com/images/P/080410753X.0...,74
508,6251,080410753X,0,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,http://images.amazon.com/images/P/080410753X.0...,74
509,6575,080410753X,8,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,http://images.amazon.com/images/P/080410753X.0...,74
510,7346,080410753X,8,The Kitchen God's Wife,Amy Tan,1992,Ivy Books,http://images.amazon.com/images/P/080410753X.0...,74


In [32]:
final_rating.shape

(1401, 9)

In [33]:
# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='ID', index='Title', values= 'Rating')

In [34]:
book_pivot

ID,254,3363,6251,6543,6575,7346,8681,8936,11601,11676,...,245963,246655,247429,247447,249628,249862,249894,250184,277427,278418
Title,,,,,,,,,,,,,,,,,,,,,
A Painted House,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,0.0,NaN,0.0
All I Really Need to Know,NaN,0.0,0.0,NaN,5.0,NaN,NaN,NaN,0.0,6.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0
Bridget Jones's Diary,0.0,NaN,NaN,0.0,4.0,NaN,0.0,NaN,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Face the Fire (Three Sisters Island Trilogy),0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,10.0,...,NaN,0.0,NaN,10.0,0.0,NaN,NaN,NaN,NaN,NaN
Harry Potter and the Order of the Phoenix (Book 5),0.0,NaN,9.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,9.0,NaN,NaN,8.0,NaN,NaN,NaN
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,10.0,8.0,5.0,NaN,10.0,NaN,NaN
STONES FROM THE RIVER,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Snow Falling on Cedars,0.0,NaN,8.0,NaN,7.0,0.0,NaN,NaN,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
The Bridges of Madison County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [35]:
book_pivot.fillna(0, inplace=True)

### Training Model

In [36]:
from scipy.sparse import csr_matrix

In [37]:
book_sparse = csr_matrix(book_pivot)

In [38]:
book_sparse

<14x418 sparse matrix of type '<class 'numpy.float64'>'
	with 333 stored elements in Compressed Sparse Row format>

In [39]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [40]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [54]:
distance, suggestion = model.kneighbors(book_pivot.iloc[13,:].values.reshape(1,-1), n_neighbors=6 )

In [155]:
distance

array([[ 0.        , 33.76388603, 35.98958183, 37.60319135, 37.68288736,
        38.09199391]])

In [55]:
suggestion

array([[13, 11,  9,  8, 10, 12]], dtype=int64)

In [63]:
book_pivot.iloc[12,:]

ID
254       0.0
3363      0.0
6251      0.0
6543      0.0
6575      0.0
         ... 
249862    0.0
249894    0.0
250184    0.0
277427    0.0
278418    0.0
Name: The Perfect Storm : A True Story of Men Against the Sea, Length: 418, dtype: float64

In [64]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['The Rainmaker', 'The Lost World', 'The Firm',
       'The Bridges of Madison County', 'The Kitchen God's Wife',
       'The Perfect Storm : A True Story of Men Against the Sea'],
      dtype='object', name='Title')


In [65]:
book_pivot.index[5]

"Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))"

In [66]:
book_names = book_pivot.index

In [67]:
book_names[2]

"Bridget Jones's Diary"

In [71]:
np.where(book_pivot.index == 'The Rainmaker'
)[0][0]

8

### finding url

In [72]:
ids = np.where(final_rating['Title'] == "The Bridges of Madison County")[0][0]


In [73]:
final_rating.iloc[ids]['Img-URL']

'http://images.amazon.com/images/P/044651652X.01.LZZZZZZZ.jpg'

In [74]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])

In [75]:
book_name[0]

Index(['The Rainmaker', 'The Lost World', 'The Firm',
       'The Bridges of Madison County', 'The Kitchen God's Wife',
       'The Perfect Storm : A True Story of Men Against the Sea'],
      dtype='object', name='Title')

In [76]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['Title'] == name)[0][0]
    ids_index.append(ids)

In [77]:
for idx in ids_index:
    url = final_rating.iloc[idx]['Img-URL']
    print(url)

http://images.amazon.com/images/P/044022165X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/034540288X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/044021145X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/044651652X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/080410753X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/006101351X.01.LZZZZZZZ.jpg


In [78]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

### Testing model

In [79]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [81]:
book_name = "The Bridges of Madison County"
recommend_book(book_name)

You searched 'The Bridges of Madison County'

The suggestion books are: 

The Lost World
The Rainmaker
The Kitchen God's Wife
The Perfect Storm : A True Story of Men Against the Sea
All I Really Need to Know
